In [ ]:
!pip install cryptography pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.6 MB/s eta 0:00:00


In [ ]:
import os
from Crypto.Cipher import AES
from Crypto.PublicKey import ElGamal
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

def generate_file_name(base_name, extension):
    return f"{base_name}_{os.urandom(4).hex()}{extension}"

def aes_encrypt_file(input_file, output_dir, aes_key):
    with open(input_file, 'rb') as f:
        data = f.read()
    cipher = AES.new(aes_key, AES.MODE_CBC)
    iv = cipher.iv
    encrypted_data = cipher.encrypt(pad(data, AES.block_size))
    _, ext = os.path.splitext(input_file)
    output_file = os.path.join(output_dir, generate_file_name("encrypted", ext))
    with open(output_file, 'wb') as f:
        f.write(iv + encrypted_data)
    print(f"AES encryption done. Encrypted file saved as: {output_file}")
    return output_file

def aes_decrypt_file(input_file, output_dir, aes_key):
    with open(input_file, 'rb') as f:
        iv = f.read(16)
        encrypted_data = f.read()
    cipher = AES.new(aes_key, AES.MODE_CBC, iv)
    decrypted_data = unpad(cipher.decrypt(encrypted_data), AES.block_size)
    _, ext = os.path.splitext(input_file)
    output_file = os.path.join(output_dir, generate_file_name("decrypted", ext))
    with open(output_file, 'wb') as f:
        f.write(decrypted_data)
    print(f"AES decryption done. Decrypted file saved as: {output_file}")
    return output_file

def rsa_encrypt_key(aes_key, public_key_file, output_dir):
    with open(public_key_file, 'rb') as f:
        public_key = RSA.import_key(f.read())
    cipher_rsa = PKCS1_OAEP.new(public_key)
    encrypted_key = cipher_rsa.encrypt(aes_key)
    encrypted_key_file = os.path.join(output_dir, generate_file_name("El-Gamal_encrypted_key", ".bin"))
    with open(encrypted_key_file, 'wb') as f:
        f.write(encrypted_key)
    print(f"El-Gamal encryption of AES key done. Key saved as: {encrypted_key_file}")
    return encrypted_key_file

def rsa_decrypt_key(encrypted_key_file, private_key_file):
    with open(private_key_file, 'rb') as f:
        private_key = RSA.import_key(f.read())
    with open(encrypted_key_file, 'rb') as f:
        encrypted_key = f.read()
    cipher_rsa = PKCS1_OAEP.new(private_key)
    aes_key = cipher_rsa.decrypt(encrypted_key)
    print("El-Gamal decryption of AES key done.")
    return aes_key

if __name__ == "__main__":
    print("Choose an operation:")
    print("1. Encrypt a file")
    print("2. Decrypt a file")
    choice = input("Enter your choice (1/2): ").strip()

    if choice == "1":
        input_file = input("Enter the full path of the file to encrypt: ").strip()
        if not os.path.isfile(input_file):
            print("Error: Input file does not exist.")
            exit(1)
        output_dir = input("Enter the directory where encrypted files should be saved: ").strip()
        if not os.path.isdir(output_dir):
            print("Error: Output directory does not exist.")
            exit(1)
        aes_key = get_random_bytes(16)
        encrypted_file = aes_encrypt_file(input_file, output_dir, aes_key)
        key = RSA.generate(2048)
        private_key_file = os.path.join(output_dir, "private.pem")
        public_key_file = os.path.join(output_dir, "public.pem")
        with open(private_key_file, 'wb') as f:
            f.write(key.export_key())
        with open(public_key_file, 'wb') as f:
            f.write(key.publickey().export_key())
        print(f"El-Gamal key pair generated. Private key: {private_key_file}, Public key: {public_key_file}")
        encrypted_key_file = rsa_encrypt_key(aes_key, public_key_file, output_dir)

    elif choice == "2":
        encrypted_file = input("Enter the full path of the encrypted file: ").strip()
        if not os.path.isfile(encrypted_file):
            print("Error: Encrypted file does not exist.")
            exit(1)
        output_dir = input("Enter the directory where decrypted files should be saved: ").strip()
        if not os.path.isdir(output_dir):
            print("Error: Output directory does not exist.")
            exit(1)
        encrypted_key_file = input("Enter the full path of the El-Gamal-encrypted AES key file: ").strip()
        if not os.path.isfile(encrypted_key_file):
            print("Error: Encrypted AES key file does not exist.")
            exit(1)
        private_key_file = input("Enter the full path of the El-Gamal private key file: ").strip()
        if not os.path.isfile(private_key_file):
            print("Error: Private key file does not exist.")
            exit(1)
        aes_key = rsa_decrypt_key(encrypted_key_file, private_key_file)
        decrypted_file = aes_decrypt_file(encrypted_file, output_dir, aes_key)

    else:
        print("Invalid choice. Exiting.")


Choose an operation:
1. Encrypt a file
2. Decrypt a file
